In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import geopandas as gpd
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt 
import shapely as shp
import os, sys
from tqdm import tqdm
sys.path.append('../scripts/')
import loadpaths
import land_cover_analysis as lca
import land_cover_visualisation as lcv
path_dict = loadpaths.loadpaths()


/home/tplas/anaconda3/envs/geo/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


adding scrub
splitting up heath
adding wetland
adding transport


## Merge main predictions in sections:

In [3]:
tile_outlines_tight = lca.load_pols('../content/landscape_character_grid/Landscape_Character_Grid_tight.shp')
assert len(tile_outlines_tight) == 1617 
col_sections = '100_grid'
assert col_sections in tile_outlines_tight.columns
arr_sections = tile_outlines_tight[col_sections].unique()
# tile_outlines_tight.head(5)

path_individual_tiles = '/media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_2023-04-24-1259_notdissolved_padding44_FGH-override/individual_tiles/'
assert len(os.listdir(path_individual_tiles)) == 1617
dir_save_merger = '/media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_2023-04-24-1259_notdissolved_padding44_FGH-override/merged_tiles_100kmsections/'


In [4]:
for section in tqdm(arr_sections):
    tiles_in_section = list(tile_outlines_tight[tile_outlines_tight[col_sections] == section]['PLAN_NO'])

    curr_dir_name = path_individual_tiles.split('/') 
    curr_dir_name = curr_dir_name[-2] if curr_dir_name[-1] != '' else curr_dir_name[-3]  # in case dir_indiv_tile_shp ends with '/'
    filename_merger = f'merged_{curr_dir_name}_section-{section}.shp'

    lca.merge_individual_shp_files(dir_indiv_tile_shp=path_individual_tiles, 
                                    delete_individual_shp_files=False,
                                    list_tile_ids_to_merge=tiles_in_section,
                                    filename=filename_merger,
                                    dir_save_merger=dir_save_merger)

  0%|          | 0/26 [00:00<?, ?it/s]

Found 95/1617 tiles in /media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_2023-04-24-1259_notdissolved_padding44_FGH-override/individual_tiles/ with tile ids
Merging 95 tiles found in /media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_2023-04-24-1259_notdissolved_padding44_FGH-override/individual_tiles/



0it [00:00, ?it/s]

95it [00:08, 11.78it/s]
  4%|▍         | 1/26 [00:30<12:37, 30.32s/it]

Found 12/1617 tiles in /media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_2023-04-24-1259_notdissolved_padding44_FGH-override/individual_tiles/ with tile ids
Merging 12 tiles found in /media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_2023-04-24-1259_notdissolved_padding44_FGH-override/individual_tiles/


...

76it [00:11,  6.67it/s]
 92%|█████████▏| 24/26 [12:58<01:29, 44.87s/it]

Found 8/1617 tiles in /media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_2023-04-24-1259_notdissolved_padding44_FGH-override/individual_tiles/ with tile ids
Merging 8 tiles found in /media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_2023-04-24-1259_notdissolved_padding44_FGH-override/individual_tiles/


8it [00:01,  7.36it/s]
 96%|█████████▌| 25/26 [13:04<00:33, 33.32s/it]

Found 10/1617 tiles in /media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_2023-04-24-1259_notdissolved_padding44_FGH-override/individual_tiles/ with tile ids
Merging 10 tiles found in /media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_2023-04-24-1259_notdissolved_padding44_FGH-override/individual_tiles/


10it [00:01,  9.16it/s]
100%|██████████| 26/26 [13:10<00:00, 30.41s/it]


## Merge detailed predictions in sections:
Difference with main is that:
- we need to merge 3 different shp files (C, D and E)
- we need to exclude no-class predictions from each
- we need to include FGH layer from main predictions 
- create new individual tiles on the fly? 

In [3]:
tile_outlines_tight = lca.load_pols('../content/landscape_character_grid/Landscape_Character_Grid_tight.shp')
# n_expected_tiles = 1617
# tile_outlines_tight = tile_outlines_tight[tile_outlines_tight['PLAN_NO'].isin(['SJ9661', 'SJ9762'])]
n_expected_tiles = 1617

assert len(tile_outlines_tight) == n_expected_tiles, f'len(tile_outlines_tight) = {len(tile_outlines_tight)}'
col_sections = '100_grid'
assert col_sections in tile_outlines_tight.columns
arr_sections = tile_outlines_tight[col_sections].unique()
# tile_outlines_tight.head(5)3

path_individual_tiles_main = '/media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_2023-04-24-1259_notdissolved_padding44_FGH-override/individual_tiles/'
path_individual_tiles_c = '/media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_2023-04-21-1335_notdissolved_clippedC_padding44/individual_tiles/'
path_individual_tiles_d = '/media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_2023-04-25-2057_notdissolved_clippedD_padding44/individual_tiles/'
path_individual_tiles_e = '/media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_2023-04-24-1216_notdissolved_clippedE_padding44/individual_tiles/'

def create_list_all_subdirs(parent_dir):
    return [os.path.join(parent_dir, x) for x in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, x))]

list_tiles_dict = {}
list_tiles_dict['main'] = create_list_all_subdirs(path_individual_tiles_main)
list_tiles_dict['c'] = create_list_all_subdirs(path_individual_tiles_c)
list_tiles_dict['d'] = create_list_all_subdirs(path_individual_tiles_d)
list_tiles_dict['e'] = create_list_all_subdirs(path_individual_tiles_e)

for k, v in list_tiles_dict.items():
    assert len(v) == n_expected_tiles, f'len(v) = {len(v)} for k = {k}'

In [4]:
df_schema = lca.create_df_mapping_labels_2022_to_80s()
dict_mapping_code_to_name = dict(zip(df_schema['code_2022'], df_schema['description_2022']))
dict_mapping_code_to_name['G2'] = dict_mapping_code_to_name['G2a']
# dict_mapping_code_to_name

adding scrub
splitting up heath
adding wetland
adding transport


In [5]:
# pols_merged.plot(column='lc_label')
# print(tile, len(tile), total_area)
pols_merged

NameError: name 'pols_merged' is not defined

In [6]:
## Step 1: merge different predictions for the same tile into one file:
path_folder_complete_tiles = '/media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_detailed-combined/individual_tiles/' 
list_tiles = list(tile_outlines_tight['PLAN_NO'])
assert len(list_tiles) == n_expected_tiles
col_label = 'lc_label'

for tile in list_tiles:
    pols_dict = {}
    for name_classifier, list_tiles_pred in list_tiles_dict.items():
        curr_tile = [x for x in list_tiles_pred if tile in x]
        assert len(curr_tile) == 1, f'len(curr_tile) = {len(curr_tile)}'
        curr_tile = curr_tile[0]
        current_pols = lca.load_pols(curr_tile)
        current_pols = current_pols.drop(columns=['class'])
        if name_classifier == 'main':  # only keep FGH layer 
            pols_dict[name_classifier] = current_pols[np.isin(current_pols[col_label].apply(lambda x: x[0]), ['F', 'G', 'H'])]
            pols_dict[name_classifier]['Class name'] = pols_dict[name_classifier][col_label].apply(lambda x: dict_mapping_code_to_name[x])
        else:  # remove no class
            pols_dict[name_classifier] = current_pols[current_pols[col_label] != '0']
    pols_merged = gpd.GeoDataFrame(pd.concat([pols_dict['main'], pols_dict['c'], pols_dict['d'], pols_dict['e']], 
                                             ignore_index=True), 
                                   crs=pols_dict['main'].crs)
    total_area = pols_merged.area.sum()
    assert total_area > 0, f'total_area = {total_area}'
    assert np.isclose(total_area, 1e6, atol=1e1), f'total_area = {total_area}'
    # assert False

    name_file = f'LCU_detailed-combined_{tile}_LC-prediction'
    if not os.path.exists(os.path.join(path_folder_complete_tiles, name_file)):
        os.makedirs(os.path.join(path_folder_complete_tiles, name_file))
    pols_merged.to_file(os.path.join(path_folder_complete_tiles, name_file, f'{name_file}.shp'))

/home/tplas/anaconda3/envs/geo/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/tplas/anaconda3/envs/geo/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/tplas/anaconda3/envs/geo/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

AssertionError: total_area = 1868126.6343917518

In [25]:
# pols_dict['d'].plot()
# tile
curr_tile

'/media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_2023-04-24-1216_notdissolved_clippedE_padding44/individual_tiles/LCU_2023-04-24-1216_SE0400_LC-prediction'

In [17]:
inds_sorted = np.argsort(pols_merged['geometry'].area)
pols_merged['area'] = pols_merged['geometry'].area
pols_merged = pols_merged.iloc[inds_sorted]
pols_merged

/home/tplas/anaconda3/envs/geo/lib/python3.7/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,lc_label,source,geometry,Class name,area
2950,D3,model prediction,"POLYGON ((404183.625 400127.000, 404183.750 40...",Bracken,0.015625
81,F3a,model prediction,"POLYGON ((404337.750 400965.875, 404337.875 40...","Wetland, Peat Bog",0.015625
68,F3a,model prediction,"POLYGON ((404351.375 400970.250, 404351.500 40...","Wetland, Peat Bog",0.015625
2204,F3a,model prediction,"POLYGON ((404220.000 400315.875, 404220.125 40...","Wetland, Peat Bog",0.015625
2580,D1a,model prediction,"POLYGON ((404019.250 400243.125, 404019.375 40...",Upland Heath,0.015625
...,...,...,...,...,...
766,D1a,model prediction,"POLYGON ((404940.875 400993.000, 404943.500 40...",Upland Heath,33325.531250
1586,D1a,model prediction,"POLYGON ((404985.000 400794.250, 404985.375 40...",Upland Heath,45893.593750
3414,D6c,model prediction,"POLYGON ((404314.125 400754.125, 404314.250 40...",Upland Heath/Blanket Peat Mosaic,190147.031250
3428,F3a,model prediction,"POLYGON ((404000.000 401000.000, 404110.500 40...","Wetland, Peat Bog",555437.953125


In [ ]:
## Step 2: merge all tiles into one file per section:
path_folder_complete_tiles = '/media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_detailed-combined/individual_tiles/'
path_folder_sections = '/media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_detailed-combined/merged_tiles_100kmsections/'

for section in tqdm(arr_sections):
    tiles_in_section = list(tile_outlines_tight[tile_outlines_tight[col_sections] == section]['PLAN_NO'])

    curr_dir_name = path_folder_complete_tiles.split('/') 
    curr_dir_name = curr_dir_name[-2] if curr_dir_name[-1] != '' else curr_dir_name[-3]  # in case dir_indiv_tile_shp ends with '/'
    # assert False, curr_dir_name
    filename_merger = f'merged_{curr_dir_name}_section-{section}.shp'

    lca.merge_individual_shp_files(dir_indiv_tile_shp=path_folder_complete_tiles, 
                                    delete_individual_shp_files=False,
                                    list_tile_ids_to_merge=tiles_in_section,
                                    filename=filename_merger,
                                    dir_save_merger=path_folder_sections)
    

  0%|          | 0/1 [00:00<?, ?it/s]

Found 2/3 tiles in /media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_detailed-combined/individual_tiles/ with tile ids
Merging 2 tiles found in /media/data-hdd/gis_pd/predictions/all_tiles_pd_notdissolved/predictions_LCU_detailed-combined/individual_tiles/



0it [00:00, ?it/s]

2it [00:00,  2.61it/s]
100%|██████████| 1/1 [00:03<00:00,  3.36s/it]


## Create 100km^2 grids for different mergers:

In [23]:
outline_pd = lca.load_pols(path_dict['pd_outline'])
outline_pd = outline_pd.iloc[0]['geometry']
all_tiles = lca.load_pols(path_dict['landscape_character_grid_path'])
all_tiles = all_tiles[all_tiles['geometry'].intersects(outline_pd)]
all_tiles = all_tiles.reset_index(drop=True)
assert len(all_tiles) == 1617

In [24]:
# all_tiles['COUNTY'].unique()
## tiles per county:
n_tiles_per_county = all_tiles.groupby('COUNTY').size()
n_tiles_per_county

COUNTY
CHESHIR    108
DERBYSH    993
GT MAN      32
SOUTHYO    194
STAFFOR    229
WESTYOR     61
dtype: int64

In [25]:
all_tiles['100_grid'] = all_tiles['PLAN_NO'].apply(lambda x: x[:3] + x[4])

In [26]:
all_tiles['100_grid'].unique()

array(['SE00', 'SE01', 'SE10', 'SE20', 'SJ96', 'SJ97', 'SJ98', 'SK05',
       'SK06', 'SK07', 'SK08', 'SK09', 'SK04', 'SK14', 'SK15', 'SK16',
       'SK17', 'SK18', 'SK19', 'SK25', 'SK26', 'SK27', 'SK28', 'SK29',
       'SK36', 'SK37'], dtype=object)

In [27]:
all_tiles.to_file('/home/thijs/repos/cnn-land-cover/content/landscape_character_grid/Landscape_Character_Grid_tight.shp')